In [1]:
"""
Train convex LassoNet models on a synthetic classification dataset using convex optimization.
"""

import sys
sys.path.append("..")

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from convex_nn import optimize
from convex_nn.utils.data import gen_classification_data

In [3]:
# Generate realizable synthetic classification problem (ie. Figure 1)
n_train = 250
n_test = 250
d = 50
hidden_units = 100
kappa = 10  # condition number

(X_train, y_train), (X_test, y_test) = gen_classification_data(123, n_train, n_test, d, hidden_units, kappa)

In [7]:
# generate feature-sparse gate vectors to use
max_patterns = 100
gates = np.random.randn(d, max_patterns)

# create feature-sparse gates
sparse_indices = np.random.choice(d, 20, replace=False)
gates[sparse_indices] = 0.

In [5]:
grelu_model, grelu_metrics = optimize(X_train, 
               y_train, 
               X_test, 
               y_test, 
               train_metrics=["accuracy"],
               test_metrics=["accuracy"],
               additional_metrics=["feature_sparsity", "active_features", "step_size"],
               U=gates, 
               formulation="grelu_lasso_net",
               reg_strength=0.05,
               backend="numpy",
               verbose=True)

INFO:convex_nn:Processing data.
INFO:convex_nn:Constructing convex model.
INFO:convex_nn:Preparing optimizer.
INFO:convex_nn:Optimizing convex model.
INFO:convex_nn:Pre-Optimization Metrics: Train Set objective: 0.5, Train Set grad_norm: 0.04219839721918106, Train Set accuracy: 0.0, Test Set accuracy: 0.0, feature_sparsity: 0.4, active_features: 30, step_size: 10.0, 


fista:   0%|          | 0/10000 [00:00<?, ?it/s]

Train Set objective: 0.5, Train Set grad_norm: 0.04219839721918106, Train Set accuracy: 0.0, Test Set accuracy: 0.0, feature_sparsity: 0.4, active_features: 30, step_size: 10.0, 


INFO:convex_nn:Termination criterion satisfied at iteration 24/10000. Exiting optimization loop.
INFO:convex_nn:Post-Optimization Metrics: Train Set objective: 0.17713644338237078, Train Set grad_norm: 9.063892921727579e-07, Train Set accuracy: 0.996, Test Set accuracy: 0.908, feature_sparsity: 0.16, active_features: 42, step_size: 10.0, 
